In [16]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [17]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [18]:
params = {"grupo":"4"}

In [19]:
r = requests.get(base_url, params=params)

In [20]:
page = BeautifulSoup(r.text, 'html.parser')

In [21]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [22]:
__EVENTVALIDATION

'/wEdAAxNHssh9XvNiCSMOikJFVy54mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc0aaExB2twtCCf/rNOZVIJX6WQ29g=='

In [23]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [24]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUBNB4IaWRmR3J1cG8CBB4IYnVzY2FQb3JkFgJmD2QWAgIDD2QWAgIBDxYCHgZhY3Rpb24FHi9kYWRvc2FiZXJ0b3MvY29uc3VsdGE/Z3J1cG89NBYOAgIPEA8WBh4NRGF0YVRleHRGaWVsZAUKTm9tZVBhZ2luYR4ORGF0YVZhbHVlRmllbGQFCVVybFBhZ2luYR4LXyFEYXRhQm91bmRnFgQeCW9ua2V5ZG93bgVDaWYoSXNFbnRlcktleShldmVudCkpe1JlZGlyZWNpb25hckFjZXNzb1JhcGlkbyh0aGlzKTtyZXR1cm4gZmFsc2U7fR4Hb25jbGljawUfUmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpOxAVBg5BY2Vzc28gUsOhcGlkbxZTZWNyZXRhcmlhcyBlIMOTcmfDo29zEVNlcnZpw6dvcyBPbi1MaW5lGFNlcnZpw6dvcyBwYXJhIGEgRW1wcmVzYRlTZXJ2acOnb3MgcGFyYSBvIENpZGFkw6NvE1NpdGVzIGRhIFByZWZlaXR1cmEVBgANL3NlY3JldGFyaWFzLxkvbGlzdGEtc2Vydmljb3Mtb24tbGluZS8yES9zZXJ2aWNvcy9lbXByZXNhES9zZXJ2aWNvcy9jaWRhZGFvHC9saXN0YS1zaXRlcy1kYS1wcmVmZWl0dXJhLzUUKwMGZ2dnZ2dnZGQCBA8PZBYCHwcFSXJldHVybiBWYWxpZGFyUGVzcXVpc2EoJ3R4dFBlc3F1aXNhVG9wbycsJ2N0bDAwJGxua1Blc3F1aXNhVG9wbycsIGV2ZW50KTtkAgYPDxYCHgRUZXh0BQ1TZXgsIDEyLzEwLzE4ZGQCBw8PFgIeCEltYWdlVXJsBRV+L2ltYWdlbS90ZW1wby9wYy5naWZkZAIIDw8WAh8JBQQyNcK6ZGQCCQ8PFgIfCQUEMTX

In [25]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [26]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [27]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [28]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [29]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [30]:
print(md5(a_tags[0].text))

979df6ebc2280f5c834cc3dff183df24


In [31]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [32]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [33]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 6/6 [00:00<00:00, 11972.32it/s]


In [34]:
databases

[{'database_id': '979df6ebc2280f5c834cc3dff183df24',
  'database_title': 'Disque Economia - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/DisqueEconomia/2018-10-08_Disque_Economia_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 8)},
 {'database_id': '203db94be8f280e2c9911f44dfcf2685',
  'database_title': 'Disque Economia - Produto - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/DisqueEconomia/2018-10-08_Disque_Economia_-_Produto_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 8)}]

In [35]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/DisqueEconomia/2018-10-08_Disque_Economia_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,5,10,14,15,16,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
test.fillna('').head()

,DATA_PESQUISA,PRECO_PESQUISADO,CODIGO_ESTABELECIMENTO,CODIGO_ESTABELECIMENTO_FILIAL,ENDERECO_ESTABELECIMENTO,NUMERO_ENDERECO_ESTABELECIMENTO,COMPLEMENTO_ENDERECO_ESTABELECIMENTO,TELEFONE_ESTABELECIMENTO,CIDADE_ESTABELECIMENTO,ESTADO_ESTABELECIMENTO,...,DESCRICAO_CATEGORIA_PRODUTO,CODIGO_PRODUTO,PRODUTO_DESCRICAO,PRODUTO_QTD_EMBALAGEM,CODIGO_TIPO_UNIDADE_MEDIDA_PRODUTO,TIPO_UNIDADE_MEDIDA_PRODUTO_DESCRICAO,TIPO_UNIDADE_MEDIDA_PRODUTO_SIGLA,CODIGO_TIPO_EMBALAGEM_PRODUTO,TIPO_EMBALAGEM_PRODUTO_DESCRICAO,TIPO_EMBALAGEM_PRODUTO_SIGLA
0,-------------,----------------,----------------------,-----------------------------,------------------------,-------------------------------,------------------------------------,------------------------,----------------------,----------------------,...,---------------------------,--------------,-----------------,---------------------,----------------------------------,-------------------------------------,---------------------------------,-----------------------------,--------------------------------,----------------------------
1,09/07/2018,5.95,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,...,1,2,Adoçante Em Po/ Acucar Light,500.0,2,GRAMA,GR,11,PACOTE,PCT
2,09/07/2018,2.45,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,...,1,4,Acucar Refinado ( + Barato ),1.0,1,KILOGRAMA,KG,11,PACOTE,PCT
3,09/07/2018,7.39,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,...,1,9,"Amido de Milho ""Maizena""",500.0,2,GRAMA,GR,4,CAIXA,CXA
4,09/07/2018,2.11,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,...,1,11,Farinha de Trigo Especial (+ Barato),1.0,1,KILOGRAMA,KG,11,PACOTE,PCT
